In [34]:
from langchain_community.llms.ollama import Ollama

model = Ollama(model='llama2')

In [35]:
import json

with open('../data/test.json', 'r') as raw_json:
    read = raw_json.read()
    parsed = json.loads(read)

old_method = parsed['src_method']
new_method = parsed['dst_method']
old_comment = parsed['src_javadoc']

In [36]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate

response_schemas = [
    ResponseSchema(name="newComment", description="the new comment"),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()


In [ ]:

from langchain_core.output_parsers import StrOutputParser

prompt1 = PromptTemplate(
    template='Please show the change sequence between the old method and the new method in one sentence. No more than 150 words. \nOld Method: ```{old_method}```\nNew Method: ```{new_method}```',
    input_variables=["old_method", "new_method"]
)
prompt2 = PromptTemplate(
    template=
    '''
    Read the JavaDoc comment below.
    Old comment: {old_comment}
    The method it belongs to has been modified according to the following content. 
    Modification: ```{modification}```
    Please infer the new comment.
    {format_instructions}''',
    input_variables=["modification", "old_comment"],
    partial_variables={"format_instructions": format_instructions},
)

chain1 = prompt1 | model | StrOutputParser()
r1 = chain1.invoke({
    'new_method': new_method,
    'old_method': old_method,
})

print('Result1: ' + r1, end='\n\n')

chain2 = (
        prompt2
        | model
        | StrOutputParser()
)
result = chain2.invoke(
    {"modification": r1, "old_comment": old_comment}
)

print(f"Result2: {result}")